In [22]:
from sqlalchemy import create_engine
import pandas as pd
import plotly.express as px
from dotenv import load_dotenv
import panel as pn
import plotly.graph_objects as go
import os

pn.extension("plotly")

In [4]:
# Variáveis do arquivo .env
load_dotenv()

True

In [5]:
# Conexão com o Banco de Dados
usuario = os.getenv("DB_USER")
senha = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
nome_do_banco = os.getenv("DB_NAME")

engine = create_engine(
    f"postgresql+psycopg2://{usuario}:{senha}@{host}:{port}/{nome_do_banco}"
)

In [ ]:
# Carregando a tabela do banco de dados
cliente = pd.read_sql("SELECT * FROM cliente", con=engine)

# Convertendo as colunas de data para datetime
cliente["data_adesao"] = pd.to_datetime(cliente["data_adesao"])
cliente["data_cancelamento"] = pd.to_datetime(cliente["data_cancelamento"])

# Agrupando por mês e contando os clientes
adesoes = (
    cliente.groupby(cliente["data_adesao"].dt.to_period("M"))
    .size()
    .reset_index(name="quantidade")
)
cancelamentos = (
    cliente.dropna(subset=["data_cancelamento"])
    .groupby(cliente["data_cancelamento"].dt.to_period("M"))
    .size()
    .reset_index(name="quantidade")
)

# Convertendo o índice de Period para datetime
adesoes["data_adesao"] = adesoes["data_adesao"].apply(lambda x: x.start_time)
cancelamentos["data_cancelamento"] = cancelamentos["data_cancelamento"].apply(lambda x: x.start_time)

# Criando os gráficos com Plotly
fig = go.Figure()

# Adicionando o gráfico de adesões
fig.add_trace(
    go.Scatter(
        x=adesoes["data_adesao"],
        y=adesoes["quantidade"],
        mode="lines+markers",
        name="Adesões",
    )
)

# Adicionando o gráfico de cancelamentos
fig.add_trace(
    go.Scatter(
        x=cancelamentos["data_cancelamento"],
        y=cancelamentos["quantidade"],
        mode="lines+markers",
        name="Cancelamentos",
    )
)

# Personalizando o layout do gráfico
fig.update_layout(
    title="Adesões e Cancelamentos de Clientes",
    xaxis_title="Data",
    yaxis_title="Quantidade de Clientes",
    legend_title="Legenda",
    template="seaborn",
)

# Exibindo o gráfico
fig.show()